# Creating Redshift Cluster using the AWS python SDK

In [ ]:
import pandas as pd
import boto3
import json
import configparser
from botocore.exceptions import ClientError

## Load DWH Params from a file

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                     = config['AWS']['key']
SECRET                  = config['AWS']['secret']

CLUSTER_TYPE            = config['CLUSTER']['cluster_type']
NUM_NODES               = config['CLUSTER']['num_nodes']
NODE_TYPE               = config['CLUSTER']['node_type']
CLUSTER_ID              = config['CLUSTER']['cluster_id']
IAM_ROLE_NAME           = config['CLUSTER']['iam_role_name']

DB_NAME                 = config['DB']['db_name']
DB_USER                 = config['DB']['db_user']
DB_PASSWORD             = config['DB']['db_password']
DB_PORT                 = config['DB']['db_port']

pd.DataFrame({
    "Param": ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_ID", "IAM_ROLE_NAME", "DB_NAME", "DB_USER", "DB_PASSWORD", "DB_PORT"],
    "Value": [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_ID, IAM_ROLE_NAME, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT]
    })

## Create clients for IAM, EC2, S3 and Redshift

In [ ]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## IAM ROLE
Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly).

In [ ]:
# Create the role
try:
    print("1.1 Creating a new IAM Role")
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
print("1.2 Attaching Policy")
iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

## Redshift Cluster
Create a RedShift Cluster. For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster).

In [ ]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_ID,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

Describe the cluster to see its status. Run this block several times until the cluster status becomes 'Available'.

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_ID)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

Take note of the cluster endpoint and role ARN. DO NOT RUN THIS unless the cluster status becomes 'Available'.

In [ ]:
DB_HOST = myClusterProps['Endpoint']['Address']
IAM_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']

config['DB']['HOST'] = DB_HOST
config['IAM_ROLE']['ARN'] = IAM_ROLE_ARN

with open('dwh.cfg', 'w') as configfile:
    config.write(configfile)

print("DB_HOST: ", DB_HOST)
print("IAM_ROLE_ARN: ", IAM_ROLE_ARN)

## Open an incoming  TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

## Make sure you can connect to the cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME)
print(conn_string)
%sql $conn_string

# Clean up your resources
DO NOT RUN THIS UNLESS YOU ARE SURE.

In [ ]:
redshift.delete_cluster( ClusterIdentifier=CLUSTER_ID,  SkipFinalClusterSnapshot=True)

Run this block several times until the cluster really deleted.

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_ID)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

Delete IAM Role.

In [ ]:
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)